In [41]:
import os
import descarteslabs as dl 
from descarteslabs.client.services.tasks import AsyncTasks, as_completed
from descarteslabs.client.services.catalog import Catalog

In [44]:
def rice_sar_stats(year, tile):

    import json
    import numpy as np
    from osgeo import gdal
    import logging
    import time
    from datetime import datetime
    catalog = Catalog()

    def arr_to_tiff(stats, meta, year, dlkey):
        
        drv = gdal.GetDriverByName('GTiff')
        nOutBands = len(stats)
        rows, cols = stats[0].shape[0], stats[0].shape[1]
        savename = year + '_' + dlkey + '.tif'
 
        dest_ds = drv.Create(savename,
                             rows, cols, nOutBands,
                             gdal.GDT_Byte,
                             options=['TILED=YES', 'COMPRESS=LZW']
                             )
        dest_ds.SetGeoTransform(meta['geoTransform'])
        dest_ds.SetProjection(meta['coordinateSystem']['wkt'].encode('utf-8'))

        for i in range(0, nOutBands):
            dest_ds.GetRasterBand(i + 1).WriteArray(stats[i])

        del dest_ds        
        
        return savename

    def calculate_stats(arr):

        vx_max = arr.max(axis=0)
        vx_min = arr.min(axis=0)
        vx_mean = arr.mean(axis=0)
        vx_med = np.ma.median(arr, axis=0)
        vx_std = arr.std(axis=0)

        stats = [vx_max, vx_min, vx_mean, vx_med, vx_std]

        return stats

    def get_imagery(year, tile, s1_pass, band):

        start = str(year) + '-01-01'
        end = str(year) + '-12-31'

        dlkey = tile['properties']['key']

        # Search for available imagery for each tile
        scenes = dl.metadata.search(
                               products=["sentinel-1:GRD"],
                               start_time=start,
                               end_time=end,
                               geom=json.dumps(tile['geometry']),
                               limit = 1000
                               )

        if s1_pass == 'DESCENDING' or s1_pass == 'ASCENDING':
            img_subset = [item for item in scenes['features'] if item['properties']['pass'] == s1_pass]
            scenes['features'] = img_subset

        ids = [scene['id'] for scene in scenes['features']]
        
        arr, meta = dl.raster.stack(
                                ids,
                                bands=band,
                                data_type='Byte',
                                dltile=dlkey,
                                )
        arr = arr[:,:,:,0]
        return arr, meta[0]

    def catalog_upload(tile, stats_list, meta, year):

        dlkey = tile['properties']['key']
        
        # generate a single geotiff with a band for each annual statistic
        savename = arr_to_tiff(stats_list, meta, year, dlkey)
        product_id  = '7294028cc01114d89a473cf055d29dc5cd5ffe88:sar_img_stats:v3'

        # write geotiff to catalog
        for tries in range(3):
            print(savename, product_id, datetime.today().isoformat())
            try:
                catalog.upload_image(savename, product_id, acquired=datetime.today().isoformat())
                print "successfully wrote %s to catalog!" % dlkey
                break
         
            except Exception as err:
                print("here")
                print err
                logging.error("Could not upload to catalog")
                waittime = 2**tries
                logging.warning("Retrying upload %s" % waittime)
                time.sleep(waittime)
                continue

    def run_analysis(year, tile):

        bands = [['vv'], ['vh']]
        passes = ['DESCENDING', 'ASCENDING', 'BOTH']
        all_stats = []

        for band in bands:
            for s1_pass in passes:
                try:
                    imagery, meta = get_imagery(year, tile, s1_pass, band)
                    arr = np.ma.masked_equal(imagery,0)
                    stats = calculate_stats(arr)
                    all_stats.append(stats)
                except:
                    tile_size = tile['properties']['tilesize']
                    nodata = np.zeros((tile_size, tile_size), dtype=np.uint8)
                    all_stats.append([nodata, nodata, nodata, nodata, nodata])
        
        # flatten the list of lists (all_stats) into a single list
        stats_list = [item for sublist in all_stats for item in sublist]
        
        catalog_upload(tile, stats_list, meta, year)
    
    run_analysis(year,tile)

In [46]:
year = '2017'
loc = 'burundi'
    
res = 20.0
pad = 0 
pixels = 1024 

aoi = dl.places.shape(loc, geom = 'low')
tiles = dl.raster.dltiles_from_shape(res, pixels, pad, aoi['geometry'])

try:
    at = AsyncTasks()
    async_function = at.create_or_get_function(
            rice_sar_stats, cpus=1, memory="6Gi", name="rice_img_stats",
            maximum_concurrency = 200,
#             image="us.gcr.io/dl-ci-cd/images/tasks/users/jason/image@sha256:05b164078a9a1415cb88d55b492ad9fc61e8babafdf2ba76aaa0c4002e347c55") 
            image='us.gcr.io/dl-ci-cd/images/tasks/public/alpha/py2/default:v2018.04.26')
    # Iterate over tiles, submit tasks
    tasks = []
    for tile in tiles['features']:

        print "TILE ID: %s" % tile['properties']['key']

        t = async_function( 
            year,
            tile,
        ) 
        tasks.append(t)

    print 'Done submitting %i tasks. Starting to collect' % len(tasks)

    for task in as_completed(tasks):

        if task.exception:
            print "Failed with exception: ", task.log
        else:
            task.result
except:
    print "tasks not submitted!"
    

TILE ID: 1024:0:20.0:35:10:-16
TILE ID: 1024:0:20.0:35:10:-15
TILE ID: 1024:0:20.0:35:11:-18
TILE ID: 1024:0:20.0:35:11:-17
TILE ID: 1024:0:20.0:35:11:-16
TILE ID: 1024:0:20.0:35:11:-15
TILE ID: 1024:0:20.0:35:11:-14
TILE ID: 1024:0:20.0:35:12:-23
TILE ID: 1024:0:20.0:35:12:-22
TILE ID: 1024:0:20.0:35:12:-21
TILE ID: 1024:0:20.0:35:12:-20
TILE ID: 1024:0:20.0:35:12:-19
TILE ID: 1024:0:20.0:35:12:-18
TILE ID: 1024:0:20.0:35:12:-17
TILE ID: 1024:0:20.0:35:12:-16
TILE ID: 1024:0:20.0:35:12:-15
TILE ID: 1024:0:20.0:35:13:-25
TILE ID: 1024:0:20.0:35:13:-24
TILE ID: 1024:0:20.0:35:13:-23
TILE ID: 1024:0:20.0:35:13:-22
TILE ID: 1024:0:20.0:35:13:-21
TILE ID: 1024:0:20.0:35:13:-20
TILE ID: 1024:0:20.0:35:13:-19
TILE ID: 1024:0:20.0:35:13:-18
TILE ID: 1024:0:20.0:35:13:-17
TILE ID: 1024:0:20.0:35:13:-16
TILE ID: 1024:0:20.0:35:14:-25
TILE ID: 1024:0:20.0:35:14:-24
TILE ID: 1024:0:20.0:35:14:-23
TILE ID: 1024:0:20.0:35:14:-22
TILE ID: 1024:0:20.0:35:14:-21
TILE ID: 1024:0:20.0:35:14:-20
TILE ID: